In [ ]:
# Manifold Markets tool for user loan statistics
# this is slow since it has to go through each market to find which ones are resolved
# use manifolio instead for just the loan amount (it is much faster!)

In [4]:
###############################
# Get user loan info from API #
###############################

####### CONFIG ##########
# Manifold username
username = 'parhizj'
# Manifold Market API key
api_key = "YOUR-API-KEY"
#########################

import requests
import time
import os
import json
from datetime import datetime

bets_url_by_username = 'https://manifold.markets/api/v0/bets?username='
market_url_by_id = 'https://manifold.markets/api/v0/market/'
json_filepath = 'manifold-loans-bets-all.json'
# store a list of resolved markets as a cache (only when isResolved is True)
market_resolved_cache_filepath = 'manifold-resolved-cache.json'

# API request headers
headers = {
    "Authorization": f"Key {api_key}",
}

In [7]:
# 1 second delay between requests
def get_username_bets_before(user_name, before=None):
    bets_before = []
    # for first run
    orig_url = bets_url_by_username + user_name
    url = orig_url
    total_count = 0
    if before is not None:
        url += '&before=' + before
    while url:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            num_bets = len(data)
            if isinstance(data, list) and num_bets > 0:
                last_bet_id = None
                for bet in data:
                    if isinstance(bet, dict) and "id" in bet:
                        before = bet["id"]
                        total_count += 1
                        bets_before.append(bet)
                time.sleep(1)  # Delay between requests to avoid hammering
                # assume the last bet is the oldest one in the JSON
                url = orig_url + '&before=' + before
            else:
                if isinstance(data, dict) and 'error' in data:
                    print("Error, data from API not a list or no data: {data}")
                url = None
            if num_bets < 1000:
                # requested a 1000, so if there is less we have hit the end of the bets
                url = None
        else:
            url = None
    return bets_before

def get_market_is_resolved(market_id):
    bets_before = []
    # for first run
    url = market_url_by_id + market_id
    response = requests.get(url, headers=headers)
    isResolved = get_market_cached(market_id)
    if isResolved:
        return True
    if response.status_code == 200:
        data = response.json()
        if 'isResolved' in data:
            isResolved = data['isResolved']
            if isResolved:
                write_market_cached(market_id)
        time.sleep(1)
    return isResolved

def get_market_cached(market_id):
    if os.path.exists(market_resolved_cache_filepath):
        with open(market_resolved_cache_filepath, 'r') as f:
            cache = json.load(f)
            if market_id in cache:
                return True
    return False

def write_market_cached(market_id):
    cached_data = []
    if os.path.exists(market_resolved_cache_filepath):
        with open(market_resolved_cache_filepath, 'r') as f:
            cached_data = json.load(f)
    else:
        print("Creating new file cache for resolved markets.")
    try:
        if market_id not in cached_data:
            cached_data.append(market_id)
            with open(market_resolved_cache_filepath, 'w') as f:
                json.dump(cached_data, f)
    except:
        return False
    

In [ ]:
bets = get_username_bets_before(username)

In [ ]:
if bets is not None:
    with open(json_filepath, 'w') as f:
        json.dump(bets, f)

In [12]:
with open(json_filepath, 'r') as f:
    all_bets = json.load(f)

bets_by_contract = {}
for bet in all_bets:
    if 'loanAmount' in bet and 'contractId' in bet:
        contractId = bet['contractId']
        contractBets = []
        if contractId in bets_by_contract:
            contractBets = bets_by_contract[contractId]
        contractBets.append(bet)
        bets_by_contract[contractId] = contractBets
totalLoans = 0
contractNetLoans = {}
open_bets = []
open_contracts = {}
for contract_id, bets in bets_by_contract.items():
    contractLoan = 0
    # get resolved status (only consider not resolved)
    isResolved = get_market_is_resolved(contract_id)
    if isResolved is False:
        open_contracts[contract_id] = bets
        for bet in bets:
            open_bets.append(bet)
            if 'loanAmount' in bet:
                contractLoan += bet['loanAmount']
        # print(f"Contract net loan for {contract_id}: {contractLoan:1.0f}")
    contractNetLoans[contract_id] = contractLoan
    totalLoans += contractLoan
    
sorted_contractNetLoans = dict(sorted(contractNetLoans.items(), key=lambda x:x[1], reverse=True))
for contract_id, loanAmount in sorted_contractNetLoans.items():
    if loanAmount >= 0.5:
        print(f"Contract net loan for {contract_id}: {loanAmount:1.0f}")

Contract net loan for w8Axni4eMBky1Mg0IxIL: 105
Contract net loan for DgT3wPneFl9GPjC9MUFw: 95
Contract net loan for cpGe9ErhdnZ5yhjzGEpO: 89
Contract net loan for SZQH87RNsqa1kplkPn5F: 75
Contract net loan for sG9xUB5jxzmUHZWkSW0d: 41
Contract net loan for fyZff32mPbXBxd5IfWeq: 40
Contract net loan for NNsQvVXnJwhYaSWNrIT0: 39
Contract net loan for ePuLMhmIC1XSHIMjDJeN: 33
Contract net loan for 6F5ZEC3VpY9WY6C3Ay7i: 32
Contract net loan for BKx2ha4LBj5uQpD4bdO2: 31
Contract net loan for Bjz06YJmRycbfbkE8anm: 30
Contract net loan for avgjyL9FAWEmeaGEbFyM: 28
Contract net loan for ieIbqFDlQlnKh7PKAQT1: 28
Contract net loan for onMAJ1OtuHSRdIXwxW2p: 25
Contract net loan for dib8XAR3WIdrAz9RgFnn: 25
Contract net loan for SVk1ZnhV2A7ZL3CEn3SB: 24
Contract net loan for 6zqhpoEPAWXalDUGwsPp: 24
Contract net loan for nNYuLGSNC3cE443aabHa: 23
Contract net loan for OlGXVx60s2I5f8UbMiF1: 22
Contract net loan for gjRZMDHlowmhOVua6RiS: 21
Contract net loan for L7pS83AxLgr0OTC2h4F5: 21
Contract net

In [13]:
print(f"# Bets (All): {len(all_bets)}")
print(f"# Bets (Not resolved): {len(open_bets)}")
print(f"Contracts (All): {len(bets_by_contract)}")
print(f"Contracts (Not resolved): {len(open_contracts)}")
print(f"Net loan amount: {totalLoans:1.0f}")

# Bets (All): 7774
# Bets (Not resolved): 2378
Contracts (All): 374
Contracts (Not resolved): 173
Net loan amount: 1696
